In [1]:
import wrf
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from netCDF4 import Dataset
import metpy as mp
import metpy.calc as mpcalc
import cartopy.feature as cf
from metpy.units import units
import windspharm as ws
from windspharm.standard import VectorWind

In [2]:
file = './wrfout_d01_2019-03-11_18:00:00'
data = Dataset(file)

In [3]:
wrf.extract_times(data, timeidx=16)

numpy.datetime64('2019-03-13T18:00:00.000000000')

In [4]:
lat1, lon1 = (33.183, -99.379) #west central TX
lat2, lon2 = (36.918, -88.123) #southwest KY
start_point = wrf.CoordPair(lat = lat1, lon= lon1)
end_point = wrf.CoordPair(lat = lat2, lon = lon2)

In [5]:
#variables
raw_geo = wrf.getvar(data, 'geopt', timeidx=16)
raw_theta = wrf.getvar(data, 'theta', timeidx=16)
raw_u, raw_v = wrf.getvar(data, 'uvmet', units='kts', timeidx=16)
raw_p = wrf.getvar(data, 'pressure', timeidx=16)
raw_w = wrf.getvar(data, 'wa', units='kts', timeidx=16)
raw_rh = wrf.getvar(data, 'rh', timeidx=16)
lat = wrf.getvar(data, 'lat')
lon = wrf.getvar(data, 'lon')

In [6]:
#calculations from lab writeup:
theta = 17.22
cx = 16.73
cy = 3.02

In [7]:
rad = np.radians(theta)
u_f = raw_u-cx
v_f = raw_v-cy
new_u = u_f*np.cos(rad)+v_f*np.sin(rad)
new_v = (u_f*-1)*np.sin(rad)+v_f*np.cos(rad)

In [8]:
#AR calculations:
AR = 99.

In [ ]:
geo = wrf.interplevel(raw_geo, raw_p, 800)
theta = wrf.interplevel(raw_theta, raw_p, 800)
u = wrf.interplevel(raw_u, raw_p, 800)
v = wrf.interplevel(raw_v, raw_p, 800)
map_lat, map_lon = wrf.latlon_coords(geo)
proj= wrf.get_cartopy(geo)

fig2 = plt.figure(figsize=(12, 9))
fig2.patch.set_facecolor('white')
ax2 = plt.axes(projection=proj)
#     #coastlines and borders:
ax2.coastlines('50m', linewidth=0.7)
ax2.add_feature(cf.STATES, linewidth=0.5)

gl = ax2.gridlines()
gl.left_labels=True
gl.top_labels=True
#geopotential height contours:
con = plt.contour(wrf.to_np(lon), wrf.to_np(lat), wrf.to_np(geo), colors='black', transform=ccrs.PlateCarree())
plt.clabel(con, inline=1, fontsize=10, fmt='%i')

dx, dy = mpcalc.lat_lon_grid_deltas(lon, lat)
fgen = mpcalc.frontogenesis(theta, u*units('kt'), v*units('kt'), dx=dx, dy=dy)*1000*100*3600*3#converted to k/100km/3hr
f_levels = np.arange(-50., 50.1, 2.5)
fgen_f = plt.contourf(wrf.to_np(lon), wrf.to_np(lat), wrf.to_np(fgen), levels=f_levels, transform=ccrs.PlateCarree(), cmap='bwr_r')
bar = plt.colorbar(fgen_f, orientation='horizontal', pad=0.05, label='FGEN (K/100km/3hr)')


geodetic = ccrs.Geodetic()
PC = ccrs.PlateCarree()
lon1_, lat1_ = PC.transform_point(lon1, lat1, geodetic)
lon2_, lat2_ = PC.transform_point(lon2, lat2, geodetic)


plt.plot([lat1_, lat2_], [lon1_, lon2_], color='red')

plt.title('20190313 18Z, 800 mb Geopotential Height and FGEN')
plt.savefig('LAB8_fig.png')

/glade/work/ekoval/conda-envs/dynamics_class/lib/python3.7/site-packages/metpy/xarray.py:1454: UserWarning: Horizontal dimension numbers not found. Defaulting to (..., Y, X) order.
  warnings.warn('Horizontal dimension numbers not found. Defaulting to '


In [ ]:
fig = plt.figure(figsize=(12,9))
fig.patch.set_facecolor('white')

ax = plt.axes()
ax.set_facecolor('dimgrey')
ax.invert_yaxis()
plt.yscale('log')
plt.yticks(np.arange(200, 1000.1, 100), ['200', '300', '400', '500', '600', '700', '800', '900', '1000'])
plt.ylim(1000, 300)
plt.ylabel('Pressure (hPa)')
plt.xlabel('Latitude, Longitude')

#W Color fill:
w_cross = wrf.vertcross(raw_w, raw_p, wrfin=data, start_point=start_point, end_point=end_point, latlon=True, meta=True) 
#xs = np.linspace(lon1, lon2, w_cross.shape[-1])
xs = np.arange(0, w_cross.shape[-1], 1)
ys = wrf.to_np(w_cross.coords['vertical'])

#Set the x-ticks to longitude labels:
x_lon = wrf.to_np(w_cross.coords['xy_loc'])
x_ticks = np.arange(x_lon.shape[0])
# coord_pairs = to_np(wspd_cross.coords["xy_loc"])
# x_ticks = np.arange(coord_pairs.shape[0])
x_labels = [pair.latlon_str(fmt="{:.2f}, {:.2f}") for pair in wrf.to_np(x_lon)]
ax.set_xticks(x_ticks[::20])
ax.set_xticklabels(x_labels[::20], fontsize=9)


w_con = ax.contourf(xs, ys, wrf.to_np(w_cross), cmap='RdBu_r', levels=np.arange(-1.5, 1.91, 0.3))
plt.colorbar(w_con, ax=ax, label='Vertial Velocity (kts)')
# XS = np.linspace(-114.29, -82.996, 

#isentrope contours:
th_cross = wrf.vertcross(raw_theta, raw_p, wrfin=data, start_point=start_point, end_point=end_point, latlon=True)
th_con = ax.contour(xs, ys, wrf.to_np(th_cross), colors='black', levels=np.arange(280, 390.1, 2), alpha=0.85, linewidths=0.8)
plt.clabel(th_con)



# #FGEN
shape = np.shape(raw_u)
fgen = np.zeros(shape)
for i in range(shape[0]):
    fgen[i] = mpcalc.frontogenesis(raw_theta[i], u[i]*units('kt'), v[i]*units('kt'), dx=dx, dy=dy)*1000*100*3600*3#converted to k/100km/3hr
fgen_cross = wrf.vertcross(fgen, raw_p, wrfin=data, start_point=start_point, end_point=end_point, meta=True)
fgen_plot = plt.contour(xs, ys, fgen_cross, colors='magenta', levels=np.arange(1.0, 16.1, 1), linewidths=1.7)


#RH contour:
rh_cross = wrf.vertcross(raw_rh, raw_p, wrfin=data, start_point=start_point, end_point=end_point, latlon=True, meta=True)
rh_con = plt.contour(xs, ys, wrf.to_np(rh_cross), colors='darkgreen', levels=[70, 85], linewidths=1.75)
plt.clabel(rh_con)

#wind vectors:
u_cross = wrf.vertcross(new_u, raw_p, wrfin=data, start_point=start_point, end_point=end_point, latlon=True, meta=True)
w_cross = wrf.vertcross(raw_w, raw_p, wrfin=data, start_point=start_point, end_point=end_point, latlon=True, meta=True)

wind = ax.quiver(xs[::5], ys[::5], wrf.to_np(u_cross[::5, ::5]), wrf.to_np(w_cross[::5, ::5]*AR))

plt.title('20190313 18Z, Isentropes, Vertical Velocity, FGEN (pink),\nRH (Green, %), and Front-Relative Winds')
plt.savefig('FGEN_vert.png')

In [ ]:
np.shape(fgen)

In [ ]:
np.shape(raw_u)